In [1]:
# paramter cell do not remove!!
# nb_parm='datalake|raw/pdf|Birddiversityanddistribution|pdf||300|150'
# nb_parm='datalake|raw/text-csv|PFW_spp_translation_table_May2024|csv||300|150'
nb_parm='llmnok'
question = 'how many bird species are in migratory?'
embed_model = "mxbai-embed-large" 
gen_model = "deepseek-r1:7b"
# collection = "Bridknowledge"

In [2]:
import sys
import os, io

sys.path.append("/home/jovyan/notebooks")
from Framework.module import Utility

## Do the task after this

### Register model

In [3]:
from dotenv import load_dotenv
import os

# Load variables from .env file
load_dotenv()

# Get your Gemini API key
api_key = os.getenv("GOOGLE_API_KEY")
# print(f"Using API Key: {api_key}")

# Then use it with LangChain
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", google_api_key=api_key)
response = llm.invoke("Tell me a joke about AI.")
print(response.content)


/opt/conda/lib/python3.11/site-packages/langchain_google_genai/chat_models.py:1037: PydanticDeprecatedSince211: Accessing the 'model_fields' attribute on the instance is deprecated. Instead, you should access this attribute from the model class. Deprecated in Pydantic V2.11 to be removed in V3.0.
  for field_name, field_info in self.model_fields.items():


Why did the AI cross the road?

Because its algorithm predicted a higher probability of finding more training data on the other side!


In [4]:
from Framework import base_tools
# filename = base_tools.read_file_from_datalake('datalake', 'Agent/column_meanings.csv')
# base_tools.upload_file_to_datalake(file_obj, 'datalake', 'upload/column_meanings.csv') 



### 🧱 Step 1: Create Files and Folders in Notebook

In [5]:
from pathlib import Path
import os

# Fix: Convert to Path object
project_root = Path("/tmp/adk_short_bot_project")
pkg_dir = project_root / "adk_short_bot"
dist_dir = project_root / "dist"

# This works now
os.makedirs(pkg_dir, exist_ok=True)


### 🧠 Step 2: Write `agent.py`

In [6]:
agent_code = """
from google.adk.agents import Agent

def count_characters(message: str) -> int:
    '''
    Count the number of characters in a message.

    Args:
        message (str): The message to count characters for

    Returns:
        int: The number of characters in the message
    '''
    return len(message)


root_agent = Agent(
    name="adk_short_bot",
    model="gemini-2.0-flash",
    description="A bot that shortens messages while maintaining their core meaning",
    instruction='''You are a message shortening assistant. Your task is to take any input message and return a more concise version while maintaining the core meaning and important details.

For each message you process, you should:
1. Count the original characters
2. Create a shortened version that is more concise
3. Count the new characters
4. Return the results in this exact format:

Original Character Count: [number]
New Character Count: [number]
New message: [shortened message]

Rules for shortening:
- Remove unnecessary words and phrases
- Use shorter synonyms where possible
- Maintain proper grammar and readability
- Keep all essential information
- Don't change the meaning of the message
- Don't use abbreviations unless they're commonly understood
''',
tools=[count_characters],
)
"""


with open(os.path.join(pkg_dir, "agent.py"), "w") as f:
    f.write(agent_code)


### 📦 Step 3: Write `setup.py`

In [7]:
setup_code = """
from setuptools import setup, find_packages

setup(
    name="adk_short_bot",
    version="0.1.0",
    description="A short bot agent using Vertex Reasoning Engines",
    packages=find_packages(),
    install_requires=[
        "google-cloud-aiplatform[adk,agent_engines]",
    ],
    python_requires=">=3.8",
)
"""

with open(os.path.join(project_root, "setup.py"), "w") as f:
    f.write(setup_code)


### ✏️ Step 4: Write Empty `__init__.py`

In [8]:
init_code = "from . import agent"
with open(os.path.join(pkg_dir, "__init__.py"), "w") as f:
    f.write(agent_code)

### 🛠️ Step 5: Build .tar.gz and .whl using Python

In [9]:
!pip install build --quiet

import subprocess

subprocess.run(["python", "-m", "build", "--outdir", os.path.join(project_root, "dist")], cwd=project_root, check=True)



[notice] A new release of pip is available: 25.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
* Creating isolated environment: venv+pip...
* Installing packages in isolated environment:
  - setuptools >= 40.8.0
* Getting build dependencies for sdist...
running egg_info
writing adk_short_bot.egg-info/PKG-INFO
writing dependency_links to adk_short_bot.egg-info/dependency_links.txt
writing requirements to adk_short_bot.egg-info/requires.txt
writing top-level names to adk_short_bot.egg-info/top_level.txt
reading manifest file 'adk_short_bot.egg-info/SOURCES.txt'
writing manifest file 'adk_short_bot.egg-info/SOURCES.txt'
* Building sdist...
running sdist
running egg_info
writing adk_short_bot.egg-info/PKG-INFO
writing dependency_links to adk_short_bot.egg-info/dependency_links.txt
writing requirements to adk_short_bot.egg-info/requires.txt
writing top-level names to adk_short_bot.egg-info/top_level.txt
reading manifest file 'adk_short_bot.egg-info/SOURCES.txt'
writing manif

  - setuptools >= 40.8.0
* Getting build dependencies for wheel...
running egg_info
writing adk_short_bot.egg-info/PKG-INFO
writing dependency_links to adk_short_bot.egg-info/dependency_links.txt
writing requirements to adk_short_bot.egg-info/requires.txt
writing top-level names to adk_short_bot.egg-info/top_level.txt
reading manifest file 'adk_short_bot.egg-info/SOURCES.txt'
writing manifest file 'adk_short_bot.egg-info/SOURCES.txt'
* Building wheel...
running bdist_wheel
running build
running build_py
creating build/lib/adk_short_bot
copying adk_short_bot/__init__.py -> build/lib/adk_short_bot
copying adk_short_bot/agent.py -> build/lib/adk_short_bot
installing to build/bdist.linux-x86_64/wheel
running install
running install_lib
creating build/bdist.linux-x86_64/wheel
creating build/bdist.linux-x86_64/wheel/adk_short_bot
copying build/lib/adk_short_bot/__init__.py -> build/bdist.linux-x86_64/wheel/./adk_short_bot
copying build/lib/adk_short_bot/agent.py -> build/bdist.linux-x86_64/w

CompletedProcess(args=['python', '-m', 'build', '--outdir', '/tmp/adk_short_bot_project/dist'], returncode=0)

In [10]:
# Upload to MinIO (optional)
for fpath in dist_dir.glob("*"):
    key = f"packages/{fpath.name}"    
    base_tools.upload_filepath_to_datalake(str(fpath), 'datalake', f'Agent/{key}')
    
    print(f"☁️ Uploaded {fpath.name} to s3://datalake/Agent/{key}")

UPLOAD FILE TO S3://datalake/Agent/packages/adk_short_bot-0.1.0-py3-none-any.whl SUCCESSFULY!
☁️ Uploaded adk_short_bot-0.1.0-py3-none-any.whl to s3://datalake/Agent/packages/adk_short_bot-0.1.0-py3-none-any.whl
UPLOAD FILE TO S3://datalake/Agent/packages/adk_short_bot-0.1.0.tar.gz SUCCESSFULY!
☁️ Uploaded adk_short_bot-0.1.0.tar.gz to s3://datalake/Agent/packages/adk_short_bot-0.1.0.tar.gz
